In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as snspd

def estimate_snr_weighted(audio_signal, base_sigma=1.2, min_segment_length=500, max_segment_length=1500, adaptive_factor=0.01):
    """
    Estimating the Signal-to-Noise Ratio (SNR) of a given speech signal using weighted adaptive smoothing,
    dynamic segment lengths, and signal power scaling for higher SNR.
    """
    if len(audio_signal.shape) > 1:
        audio_signal = np.mean(audio_signal, axis=1)

    normalized_signal = audio_signal / np.max(np.abs(audio_signal))
    power_signal = np.mean(normalized_signal ** 2)
    max_signal_value = np.max(np.abs(normalized_signal))
    adaptive_noise_threshold = adaptive_factor * max_signal_value

    smoothed_signal = np.zeros_like(normalized_signal)
    total_samples = len(normalized_signal)
    pos = 0

    while pos < total_samples:
        segment_length = np.random.randint(min_segment_length, max_segment_length)
        end = min(pos + segment_length, total_samples)
        segment = normalized_signal[pos:end]
        segment_power = np.abs(segment).mean()

        if segment_power < adaptive_noise_threshold:
            local_sigma = base_sigma + 3
        else:
            local_sigma = base_sigma

        smoothed_signal[pos:end] = gaussian_filter1d(segment, sigma=local_sigma)
        pos = end

    noise = normalized_signal - smoothed_signal
    power_noise = np.mean(noise ** 2)
    snr = 10 * np.log10(power_signal / power_noise)

    return snr

def calculate_snr_for_folder(folder_path, output_excel_file):
    """
    Calculates the SNR for all audio files in a given folder and saves the results in an Excel file.

    Args:
    folder_path (str): Path to the folder containing audio files.
    output_excel_file (str): Path to the output Excel file to save the results.
    """
    results = []
    audio_extensions = ('.wav', '.flac', '.ogg')

    for filename in os.listdir(folder_path):
        if filename.endswith(audio_extensions):
            file_path = os.path.join(folder_path, filename)
            try:
                # Reading the audio file
                audio_signal, fs = sf.read(file_path)

                # Estimating SNR
                snr_estimated = estimate_snr_weighted(audio_signal)

                # Appending the result
                results.append({'Filename': filename, 'SNR (dB)': snr_estimated})
                print(f"Processed {filename}: SNR = {snr_estimated:.2f} dB")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Saving results to Excel file
    df = pd.DataFrame(results)
    df.to_excel(output_excel_file, index=False)
    print(f"Results saved to {output_excel_file}")

from google.colab import drive
drive.mount('/content/drive')

# Update the folder_path to point to the correct location on Google Drive
folder_path = '/content/drive/My Drive//denois_recodings'
output_excel_file = '/content/drive/My Drive/snr_denois_results.xlsx'



# Calculating SNR for all files in the folder and saving to Excel
calculate_snr_for_folder(folder_path, output_excel_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed 3_jackson_9.wav: SNR = 21.16 dB
Processed 3_nicolas_20.wav: SNR = 14.15 dB
Processed 3_nicolas_22.wav: SNR = 15.96 dB
Processed 3_nicolas_17.wav: SNR = 15.84 dB
Processed 3_nicolas_0.wav: SNR = 17.18 dB
Processed 3_nicolas_12.wav: SNR = 15.12 dB
Processed 3_nicolas_21.wav: SNR = 15.51 dB
Processed 3_nicolas_10.wav: SNR = 12.73 dB
Processed 3_jackson_7.wav: SNR = 16.22 dB
Processed 3_jackson_8.wav: SNR = 18.34 dB
Processed 3_nicolas_23.wav: SNR = 17.57 dB
Processed 3_nicolas_1.wav: SNR = 17.99 dB
Processed 3_nicolas_14.wav: SNR = 15.16 dB
Processed 3_nicolas_13.wav: SNR = 18.67 dB
Processed 3_nicolas_11.wav: SNR = 15.47 dB
Processed 5_theo_17.wav: SNR = 11.03 dB
Processed 6_jackson_9.wav: SNR = 14.33 dB
Processed 5_theo_31.wav: SNR = 9.77 dB
Processed 5_theo_35.wav: SNR = 9.26 dB
Processed 5_theo_48.wav: SNR = 9.39 dB
Processed 6_nicolas_13.wav: SNR 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')